In [172]:
import numpy as np # 1.16.2
import pandas as pd # 0.25.1
import sklearn.metrics # 0.21.2

In [173]:
def read_data(path):
    df = pd.read_csv(path, header=None)
    return df.to_numpy()

def split_different_models(data):
    predictions1, predictions2, predictions3 = [], [], []
    fixed_sums = 0
    for row in data:
        if np.sum(row[2:7]) != 1:
            row[np.random.choice(5) + 2] += 1 - np.sum(row[2:7])
            fixed_sums += 1
        if row[0] == 'model_id_1':
            predictions1.append(row[2:])
        elif row[0] == 'model_id_2':
            predictions2.append(row[2:])
        elif row[0] == 'model_id_3':
            predictions3.append(row[2:])
        else:
            raise ValueError(f"Unexpected model {row[0]}")
        
    print(f"Fixed {fixed_sums} probablity sums to be equal to 1")
    return map(lambda x: np.array(x, dtype='float64'), [predictions1, predictions2, predictions3])

data = read_data('student_dataset.csv')
predictions1, predictions2, predictions3 = split_different_models(data)  
print(f'{len(predictions1)} predictions for class 1')
print(f'{len(predictions2)} predictions for class 2')
print(f'{len(predictions3)} predictions for class 3')

Fixed 49820 probablity sums to be equal to 1
100000 predictions for class 1
100000 predictions for class 2
100000 predictions for class 3


In [174]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def calculate_prediction_accuracy_with_random_choice(prediction):
    model_classes = [np.random.choice(5, 1, p=row[:5]) for row in prediction]
    return sklearn.metrics.accuracy_score(prediction[:, -1], model_classes) 

def calculate_prediction_accuracy_with_argmax(prediction):
    model_classes = [np.argmax(row[:5]) for row in prediction]
    return sklearn.metrics.accuracy_score(prediction[:, -1], model_classes) 

def calculate_prediction_accuracy_with_softmax(prediction):
    model_classes = [np.random.choice(5, 1, p=softmax(row[:5])) for row in prediction]
    return sklearn.metrics.accuracy_score(prediction[:, -1], model_classes)

def calculate_accuracy(predictions1, predictions2, predictions3, f):
    print(f"Accuracy of the first model is {f(predictions1)}")
    print(f"Accuracy of the second model is {f(predictions2)}")
    print(f"Accuracy of the third model is {f(predictions3)}")

In [175]:
calculate_accuracy(predictions1, predictions2, predictions3, calculate_prediction_accuracy_with_random_choice)

Accuracy of the first model is 0.54818
Accuracy of the second model is 0.35693
Accuracy of the third model is 0.59298


In [ ]:
print("Applying softmax")
calculate_accuracy(predictions1, predictions2, predictions3, calculate_prediction_accuracy_with_softmax)

Applying softmax
Accuracy of the first model is 0.28457


In [ ]:
print("Choosing class by argmax")
calculate_accuracy(predictions1, predictions2, predictions3, calculate_prediction_accuracy_with_argmax)

В первую очередь стоит заметить, что для первой модели вероятности классов это 0, 0.5 или 1. Для второй модели вероятности могут быть абсолютно любыми, и зачастую есть две вероятности, которые очень близки друг к другу. В третьей модели вероятности это 0 или 1.

Таким образом, для первой и третьей модели не стоит вопрос как выбрать класс. Для второй же модели можно пробовать различные варианты.

Я пробовал выбирать класс пропорционально вероятностям, применять softmax, а потом выбирать с вероятностями и просто брать наибольшую вероятность. Именно наибольшая вероятность показала наибольший результат среди всех моделей. 
По второй модели у меня возникло ощущение, что она как будто недотренирована, уж слишком часто там получаются близкие вероятности.


Кроме того, я пытался анализировать данные несколькими другими способами. Я строил confusion_matrix и смотрел точности для каждого класса по отдельности, но там не нашел интересных закономерностей.

Помимо accuracy я считал и другие метрики (precission, recall, f1), но они были очень похожи на accuracy и не давали новой информации.